<a href="https://colab.research.google.com/github/juandag97/Python-Notebooks/blob/master/JD_Reinforcement_Learning_para_problemas_de_Stock_Market_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%208%20-%20Reinforcement%20Learning%20para%20problemas%20de%20Stock%20Market%20Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno

In [1]:
!pip uninstall -y tensorflow 
!pip install tensorflow-gpu==2.0.0.alpha0
%tensorflow_version 2.x

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 332.5 MB 30 kB/s 
     |████████████████████████████████| 3.0 MB 29.7 MB/s 
     |████████████████████████████████| 411 kB 66.3 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 


In [2]:
!pip install pandas-datareader

In [3]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   21.1.0
arviz                         0.11.4
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.1.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4                           0.0.1
CacheC

## Paso 2: Importar las dependencias del proyecto

In [4]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [5]:
# tf.__version__
# tf.version.VERSION

## Paso 3: Construir la red neuronal de la AI del Trader 

In [6]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Manten, Compra, Vende
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Paso 4: Pre procesado del dataset

### Definir las funciones adicionales

#### Sigmoide

In [7]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Función de formato de precios

In [8]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Carga del dataset

In [9]:
!pip install yfinance

In [10]:
import yfinance as yf
import datetime

def dataset_loader(stock_name):
  
  #Complete the dataset loader function

  start_date = datetime.datetime(2011,10,1)
  end_date = datetime.datetime(2021,10,1)
  dataset = yf.Ticker(stock_name)
  data = dataset.history(start=start_date, end= end_date)
  # dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(data.index[0]).split()[0]
  end_date = str(data.index[-1]).split()[0]
  
  close = data['Close']

  return close

In [11]:
dataset = yf.Ticker("AAPL")
print(dataset.history(periodo="max"))

                  Open        High  ...  Dividends  Stock Splits
Date                                ...                         
2021-09-13  150.630005  151.419998  ...          0             0
2021-09-14  150.350006  151.070007  ...          0             0
2021-09-15  148.559998  149.440002  ...          0             0
2021-09-16  148.440002  148.970001  ...          0             0
2021-09-17  148.820007  148.820007  ...          0             0
2021-09-20  143.800003  144.839996  ...          0             0
2021-09-21  143.929993  144.600006  ...          0             0
2021-09-22  144.449997  146.429993  ...          0             0
2021-09-23  146.649994  147.080002  ...          0             0
2021-09-24  145.660004  147.470001  ...          0             0
2021-09-27  145.470001  145.960007  ...          0             0
2021-09-28  143.250000  144.750000  ...          0             0
2021-09-29  142.470001  144.449997  ...          0             0
2021-09-30  143.660004  1

### State creator

In [12]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Cargar una divisa de mercado

In [13]:
stock_name = "AAPL"
data = dataset_loader(stock_name)
data.shape

(2516,)

In [14]:
data.head()

Date
2011-10-03    11.487059
2011-10-04    11.422664
2011-10-05    11.598989
2011-10-06    11.572004
2011-10-07    11.339871
Name: Close, dtype: float64

## Paso 5: Entrenar la AI Trader

### Configurar los hyper parámetros

In [15]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Definir el modelo del AI Trader

In [16]:
trader = AI_Trader(window_size)

In [17]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Bucle de entrenamiento

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episodio: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Compra
      trader.inventory.append(data[t])
      print("AI Trader compró: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Vende
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader vendió: ", stocks_price_format(data[t]), " Beneficio: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("BENEFICIO TOTAL: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

Episodio: 1/1000


  0%|          | 0/2515 [00:00<?, ?it/s]

AI Trader compró:  $ 11.422664
AI Trader compró:  $ 11.598989
AI Trader vendió:  $ 11.572004  Beneficio: $ 0.149341
AI Trader compró:  $ 12.333103
AI Trader vendió:  $ 12.524455  Beneficio: $ 0.925466
AI Trader vendió:  $ 12.947935  Beneficio: $ 0.614832
AI Trader compró:  $ 12.047307
AI Trader compró:  $ 12.442884
AI Trader vendió:  $ 12.284348  Beneficio: $ 0.237041
AI Trader vendió:  $ 12.409766  Beneficio: - $ 0.033118
AI Trader compró:  $ 12.158932
AI Trader vendió:  $ 12.186527  Beneficio: $ 0.027596
AI Trader compró:  $ 12.456993
AI Trader vendió:  $ 11.794319  Beneficio: - $ 0.662674


  1%|▏         | 33/2515 [00:00<01:01, 40.41it/s]

AI Trader compró:  $ 11.573229
AI Trader vendió:  $ 11.497487  Beneficio: - $ 0.075742
AI Trader compró:  $ 11.545632


...

AI Trader vendió:  $ 129.446060  Beneficio: $ 2.506248


 98%|█████████▊| 2475/2515 [07:06<00:07,  5.70it/s]

AI Trader compró:  $ 147.139542
AI Trader compró:  $ 146.730164


 98%|█████████▊| 2477/2515 [07:06<00:07,  5.37it/s]

AI Trader vendió:  $ 146.839996  Beneficio: - $ 0.299545
AI Trader vendió:  $ 146.139999  Beneficio: - $ 0.590164
